In [1]:
import sys
import tensorflow as tf


physical_device = tf.config.experimental.list_physical_devices('GPU')
print(f'Device found : {physical_device}')

if len(physical_device) is 0:
    print('No GPUs found!')
    sys.exit(-1)

Device found : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import os
import numpy as np


# These are specific to ShanghaiTech
IMAGE_FOLDER_NAME = 'images'
GT_FOLDER_NAME = 'ground-truth'
DMAP_FOLDER_NAME = 'density-map'
CROWDSEG_FOLDER_NAME = 'crowd-segment'

IMAGE_PREFIX = 'IMG_'
GT_PREFIX = 'GT_IMG_'
DMAP_PREFIX = 'DMAP_'
CROWDSEG_PREFIX = 'CROWDSEG_'

# Edit this line if your relative path is different
data_folders = ['data/ShanghaiTech/part_B/' + folder for folder in ['train_data', 'test_data']]

# Create necessary folders
for folder in data_folders:
    os.makedirs(os.path.join(folder, DMAP_FOLDER_NAME), exist_ok=True)
    os.makedirs(os.path.join(folder, CROWDSEG_FOLDER_NAME), exist_ok=True)

In [3]:
import cv2
import utils
from scipy.io import loadmat
from tqdm import tqdm


save_dmap = False
save_crowdseg = False

for folder in data_folders:
    for file in tqdm(os.listdir(os.path.join(folder, IMAGE_FOLDER_NAME))):
        img_path = os.path.join(folder, IMAGE_FOLDER_NAME, file)
        gt_path = img_path.replace(IMAGE_PREFIX, GT_PREFIX).replace('jpg', 'mat').replace(IMAGE_FOLDER_NAME, GT_FOLDER_NAME)
        dmap_path = img_path.replace(IMAGE_PREFIX, DMAP_PREFIX).replace('jpg', 'npy').replace(IMAGE_FOLDER_NAME, DMAP_FOLDER_NAME)
        crowdseg_path = img_path.replace(IMAGE_PREFIX, CROWDSEG_PREFIX).replace('jpg', 'npy').replace(IMAGE_FOLDER_NAME, CROWDSEG_FOLDER_NAME)

        image = cv2.imread(img_path)
        points = loadmat(gt_path)['image_info'][0, 0][0, 0][0]
        
        dmap = utils.gen_density_map_gaussian(image, points)
        crowdseg = utils.gen_crowdseg(image, dmap)

        if save_dmap:
            np.save(dmap_path, dmap)
        
        if save_crowdseg:
            np.save(crowdseg_path, crowdseg)

100%|██████████| 316/316 [00:20<00:00, 15.65it/s]


In [1]:
import cv2
import matplotlib.pyplot as plt


# Show the last-run density maps and segmented images.
plt.imshow(image)
plt.show()

plt.imshow(dmap)
plt.show()

plt.imshow(crowdseg)
plt.show()